In [4]:
import collections

# ----------------------------------------------------
# --- 1. BASE DE CONOCIMIENTO (HECHOS) ---
# Define la red de transporte: {Origen: [(Destino, Línea, Tiempo_minutos)]}
# ----------------------------------------------------
CONEXIONES = {
    "A": [("B", "L1", 5), ("D", "L2", 8)],
    "B": [("A", "L1", 5), ("C", "L1", 6), ("E", "L3", 10)],
    "C": [("B", "L1", 6), ("F", "L4", 7)],
    "D": [("A", "L2", 8), ("G", "L2", 4)],
    "E": [("B", "L3", 10), ("F", "L3", 5)],
    "F": [("C", "L4", 7), ("E", "L3", 5), ("Z", "L4", 12)], # Z es el destino
    "G": [("D", "L2", 4)],
    "Z": [("F", "L4", 12)]
}

# ----------------------------------------------------
# --- 2. MOTOR DE INFERENCIA Y REGLAS LÓGICAS (Funciones) ---
# ----------------------------------------------------

def regla_buscar_rutas(origen, destino):
    """
    Regla Lógica y Motor de Inferencia: Usa BFS para encontrar rutas.
    El estado en la cola es: (estación_actual, ruta_tomada, tiempo_total, linea_anterior, transferencias)
    """
    
    # Usamos deque para la búsqueda en amplitud (BFS).
    cola = collections.deque([(origen, [origen], 0, None, 0)])
    rutas_encontradas = []
    
    # Conjunto para evitar ciclos. El estado clave es (estación, línea_actual).
    visitados = set() 

    while cola:
        actual, ruta, tiempo, linea_ant, transfers = cola.popleft()

        # Evitar re-visitar estados poco óptimos (si ya llegué a esta estación por la misma línea con menos tiempo)
        if (actual, linea_ant) in visitados and actual != destino:
             continue
        visitados.add((actual, linea_ant))

        if actual == destino:
            rutas_encontradas.append({
                "ruta": ruta,
                "tiempo_total": tiempo,
                "transferencias": transfers
            })
            continue 
            
        if actual in CONEXIONES:
            for siguiente, linea_nueva, tiempo_tramo in CONEXIONES[actual]:
                
                # Regla Lógica: Contar Transferencia (si la línea cambia)
                nueva_transfers = transfers
                if linea_ant is not None and linea_nueva != linea_ant:
                    nueva_transfers += 1
                
                # Regla Lógica: Actualizar Estado
                nueva_ruta = ruta + [siguiente]
                nuevo_tiempo = tiempo + tiempo_tramo
                
                # Regla de Poda (Límite): No buscar rutas con más de 2 transferencias.
                if nueva_transfers <= 2:
                    cola.append((siguiente, nueva_ruta, nuevo_tiempo, linea_nueva, nueva_transfers))
                    
    return rutas_encontradas

def motor_principal(origen, destino):
    """
    Función Principal: Aplica la Regla de Optimización para definir la "mejor" ruta.
    """
    print(f"## 🗺️ Buscando la mejor ruta de **{origen}** a **{destino}**")
    print("-----------------------------------------")
    
    todas_las_rutas = regla_buscar_rutas(origen, destino)

    if not todas_las_rutas:
        print("❌ No se pudo encontrar una ruta válida entre las estaciones.")
        return

    # Regla de Optimización:
    # 1. Mínimas transferencias.
    # 2. Mínimo tiempo total (como desempate).
    
    mejor_ruta = min(
        todas_las_rutas, 
        key=lambda r: (r['transferencias'], r['tiempo_total'])
    )

    print("✅ **MEJOR RUTA ENCONTRADA**")
    print(f"* **Ruta:** {' -> '.join(mejor_ruta['ruta'])}")
    print(f"* **Tiempo Estimado:** {mejor_ruta['tiempo_total']} minutos")
    print(f"* **Transferencias:** {mejor_ruta['transferencias']}")
    print("\n---")
    
    # Mostrar otras opciones para completar el desarrollo del sistema experto
    otras_rutas = [r for r in todas_las_rutas if r != mejor_ruta]
    if otras_rutas:
        print("### Otras Opciones Encontradas:")
        otras_rutas.sort(key=lambda r: r['tiempo_total'])
        for i, r in enumerate(otras_rutas[:3]): 
             print(f"{i+1}. Ruta: {' -> '.join(r['ruta'])} | Tiempo: {r['tiempo_total']} min | Transfers: {r['transferencias']}")

# ----------------------------------------------------
# --- 3. EJECUCIÓN DEL SISTEMA ---
# ----------------------------------------------------

# Define el punto de inicio y el destino
PUNTO_A = "A"
PUNTO_B = "Z" 

# Ejecuta el motor
motor_principal(PUNTO_A, PUNTO_B)

## 🗺️ Buscando la mejor ruta de **A** a **Z**
-----------------------------------------
✅ **MEJOR RUTA ENCONTRADA**
* **Ruta:** A -> B -> C -> F -> Z
* **Tiempo Estimado:** 30 minutos
* **Transferencias:** 1

---
### Otras Opciones Encontradas:
1. Ruta: A -> B -> E -> F -> Z | Tiempo: 32 min | Transfers: 2
